In [1]:
import sys
sys.path.insert(0, '/home/serhii/AccountEye/')

import ImagePreprocessing.ImageProcessing as IP
import Experiments.Experiment as E
import Experiments.Models as M
import Experiments.Data as D

from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import confusion_matrix 
from sklearn.svm import SVC

from tensorflow.keras.utils import multi_gpu_model
import tensorflow.keras as K
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np
import pickle
import cv2

In [2]:
csvPaths = ['Images/Labels/Clean_Augmented.csv', 
            'Images/Labels/YouTube_Augmented.csv',
            'Images/Labels/NewBalanced_Augmented.csv'] 
            
imgPath = 'Images/Augmented/'

#csvPaths = ['/home/serhii/AccountEyeTest/Images/Augmented/NewBalanced_Blur+Augmentation.csv',]
#imgPath = '/opt/AccountEye/Images/Preproc/'

trainPart = 0.7
testPart = 0.2
batchSize = 256
numClasses = 20
epochs = 10
rows, cols, channels = 48, 32, 3
input_shape = (rows, cols, channels)

In [4]:
(xTrain, yTrainO), (xTest, yTestO), (xValid, yValidO) = D.LoadData(csvPaths, imgPath, rows, cols, channels=channels,
                                                              trainPart=trainPart, testPart=testPart, balanceDS=True, labelRow='20_Classes')


In [5]:
print(xTrain.shape)
print(xTest.shape)
print(xValid.shape)
print(yTrainO.shape)
print(yTestO.shape)
print(yValidO.shape)

(11312, 48, 32, 3)
(3231, 48, 32, 3)
(1617, 48, 32, 3)
(11312,)
(3231,)
(1617,)


# Data

In [7]:
yTrain = K.utils.to_categorical(yTrainO, numClasses)
yTest = K.utils.to_categorical(yTestO, numClasses)
yValid = K.utils.to_categorical(yValidO, numClasses)

In [8]:
xTrainHOG = np.zeros((xTrain.shape[0], 540))
for i in range(0, xTrain.shape[0]):
    xTrainHOG[i] = IP.HoG(xTrain[i]).flatten()

In [9]:
xTestHOG = np.zeros((xTest.shape[0], 540))
for i in range(0, xTest.shape[0]):
    xTestHOG[i] = IP.HoG(xTest[i]).flatten()

In [10]:
xValidHOG = np.zeros((xValid.shape[0], 540))
for i in range(0, xValid.shape[0]):
    xValidHOG[i] = IP.HoG(xValid[i]).flatten()

In [12]:
xTrain = xTrain/255
xTest = xTest/255
xValid = xValid/255

In [13]:
xTrainFL = xTrain.reshape(xTrain.shape[0], -1)
xTestFL = xTest.reshape(xTest.shape[0], -1)
xValidFL = xValid.reshape(xValid.shape[0], -1)

# SVM

In [14]:
fileSVM = 'SVM_Release.sav'

In [15]:
# Use for create model
modelSVM = SVC(max_iter=-1, verbose=True)

In [16]:
# Use for load model
if modelSVM is None:
    modelSVM = pickle.load(open(fileSVM, 'rb'))

In [17]:
modelSVM.fit(xTrainFL, yTrainO)

In [18]:
pickle.dump(modelSVM, open(fileSVM, 'wb'))

In [19]:
accTrain = modelSVM.score(xTrainFL, yTrainO) 
accTest = modelSVM.score(xTestFL, yTestO) 
accValid = modelSVM.score(xValidFL, yValidO) 

In [20]:
print(accTrain)
print(accTest)
print(accValid)

# SVM HOG

In [21]:
fileSVM_HOG = 'SVM_HOG_Release.sav'

In [22]:
# Use for create model
modelSVM_HOG = SVC(max_iter=-1, verbose=True)

In [23]:
# Use for load model
if modelSVM_HOG is None:
    modelSVM_HOG = pickle.load(open(fileSVM_HOG, 'rb'))

In [24]:
modelSVM_HOG.fit(xTrainHOG, yTrainO)

In [25]:
pickle.dump(modelSVM_HOG, open(fileSVM, 'wb'))

In [26]:
accTrain = modelSVM_HOG.score(xTrainHOG, yTrainO) 
accTest = modelSVM_HOG.score(xTestHOG, yTestO) 
accValid = modelSVM_HOG.score(xValidHOG, yValidO) 

In [27]:
print(accTrain)
print(accTest)
print(accValid)

In [28]:
predictTrainSVM_HOG = modelSVM.predict(xTrainHOG)
predictTestSVM_HOG = modelSVM.predict(xTestHOG)
predictValidSVM_HOG = modelSVM.predict(xValidHOG)

In [29]:
# creating a confusion matrix 
sns.heatmap(confusion_matrix(yTrainO, predictTrainSVM_HOG))

In [30]:
sns.heatmap(confusion_matrix(yTestO, predictTestSVM_HOG))

In [31]:
sns.heatmap(confusion_matrix(yValidO, predictValidSVM_HOG))

# K-Neighbors

In [32]:
fileKNN = 'KNN_Release.sav'

In [33]:
# Use for create model
modelKNN = KNN(n_neighbors = 7, n_jobs=10)

In [34]:
# Use for load model
if modelKNN is None:
    modelKNN = pickle.load(open(fileKNN, 'rb'))

In [35]:
modelKNN.fit(xTrainHOG, yTrainO) 

In [36]:
pickle.dump(modelKNN, open(fileKNN, 'wb'))

In [37]:
accTrain = modelKNN.score(xTrainHOG, yTrainO) 
accTest = modelKNN.score(xTestHOG, yTestO) 
accValid = modelKNN.score(xValidHOG, yValidO) 

In [38]:
print(accTrain)
print(accTest)
print(accValid)

In [39]:
# creating a confusion matrix 
predictTrainKNN = modelKNN.predict(xTrain)
predictTestKNN = modelKNN.predict(xTest)
predictValidKNN = modelKNN.predict(xValid)

In [40]:
sns.heatmap(confusion_matrix(yTrainO, predictTrainKNN))

In [41]:
sns.heatmap(confusion_matrix(yTestO, predictTestKNN))

In [42]:
sns.heatmap(confusion_matrix(yValidO, predictValidKNN))

# VGG

In [43]:
modelVGG = M.CreateVGGModel(input_shape, numClasses)

In [44]:
modelVGG = multi_gpu_model(modelVGG, gpus=2)

In [45]:
M.CompileModel(modelVGG)

In [47]:
M.FitModel(modelVGG, xTrain, yTrain, xTest, yTest, epochs=20, batch_size=batchSize, modelName='VGG_Release')

In [48]:
print('trn', M.EvaluateModel(modelVGG, xTrain, yTrain))
print('tes', M.EvaluateModel(modelVGG, xTest, yTest))
print('val', M.EvaluateModel(modelVGG, xValid, yValid))

Test loss: 0.0015864503934604345
Test accuracy: 0.9992927864214993
trn None
Test loss: 0.06706133615576358
Test accuracy: 0.9835964097802538
tes None
Test loss: 0.043824135445180164
Test accuracy: 0.9882498453927026
val None
Test loss: 0.5658007732268059
Test accuracy: 0.9306930693069307
cln None


In [49]:
M.SaveModel(modelVGG,'VGG_Release.hdf5')

# ResNet

In [50]:
modelRes = M.CreateResNetModel(input_shape, numClasses)

In [51]:
modelRes = multi_gpu_model(modelRes, gpus=2)

In [52]:
M.CompileModel(modelRes)

In [54]:
M.FitModel(modelRes, xTrain, yTrain, xTest, yTest, epochs=5, batch_size=batchSize, modelName='Res_Release')

In [55]:
print('trn', M.EvaluateModel(modelRes, xTrain, yTrain))
print('tes', M.EvaluateModel(modelRes, xTest, yTest))
print('val', M.EvaluateModel(modelRes, xValid, yValid))

Test loss: 1.544289961771189e-05
Test accuracy: 1.0
trn None
Test loss: 0.097282465076603
Test accuracy: 0.9792633859486227
tes None
Test loss: 0.0689836024085259
Test accuracy: 0.9808286951144094
val None
Test loss: 0.5229637571638174
Test accuracy: 0.9222772277227723
cln None


In [56]:
M.SaveModel(modelRes,'Res_Release.hdf5')